In [1]:
pip install dask[dataframe]

In [2]:
pip install dask-ml

In [3]:
import dask.dataframe as dd
import dask.array as da
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from scipy.stats import chi2_contingency
from dask_ml.model_selection import train_test_split



In [4]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from dask_ml.wrappers import ParallelPostFit
from dask_ml.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [5]:
from dask_ml.linear_model import LogisticRegression

In [6]:
from dask_ml.preprocessing import DummyEncoder


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:

# Read the CSV files using Dask and specify dtypes
train_df = dd.read_csv('/content/drive/MyDrive/microsoft_projData/GUIDE_Train.csv', dtype={'ActionGranular': 'object',
       'ActionGrouped': 'object',
       'AntispamDirection': 'object',
       'ResourceType': 'object',
       'Roles': 'object',
       'ThreatFamily': 'object'})
test_df = dd.read_csv('/content/drive/MyDrive/microsoft_projData/GUIDE_Test.csv', dtype={'ActionGranular': 'object',
       'ActionGrouped': 'object',
       'AntispamDirection': 'object',
       'ResourceType': 'object',
       'Roles': 'object',
       'ThreatFamily': 'object'})



In [23]:
len(train_df), len(test_df)

(9516837, 4147992)

In [9]:
frac1 = 50000 / len(train_df)

0.005253846419771611

In [24]:
frac2 = 8000 / len(test_df)
frac2

0.0019286440282430632

In [25]:
train_df = train_df.sample(frac=0.005, random_state=42)
test_df = test_df.sample(frac=0.002, random_state=42)

In [26]:

train_df.shape[0].compute(), train_df.shape[1]

(47582, 45)

In [27]:
test_df.shape[0].compute(), test_df.shape[1]

(8297, 46)

In [28]:
train_df.columns

Index(['Id', 'OrgId', 'IncidentId', 'AlertId', 'Timestamp', 'DetectorId',
       'AlertTitle', 'Category', 'MitreTechniques', 'IncidentGrade',
       'ActionGrouped', 'ActionGranular', 'EntityType', 'EvidenceRole',
       'DeviceId', 'Sha256', 'IpAddress', 'Url', 'AccountSid', 'AccountUpn',
       'AccountObjectId', 'AccountName', 'DeviceName', 'NetworkMessageId',
       'EmailClusterId', 'RegistryKey', 'RegistryValueName',
       'RegistryValueData', 'ApplicationId', 'ApplicationName',
       'OAuthApplicationId', 'ThreatFamily', 'FileName', 'FolderPath',
       'ResourceIdName', 'ResourceType', 'Roles', 'OSFamily', 'OSVersion',
       'AntispamDirection', 'SuspicionLevel', 'LastVerdict', 'CountryCode',
       'State', 'City'],
      dtype='object')

In [29]:
test_df.columns

Index(['Id', 'OrgId', 'IncidentId', 'AlertId', 'Timestamp', 'DetectorId',
       'AlertTitle', 'Category', 'MitreTechniques', 'IncidentGrade',
       'ActionGrouped', 'ActionGranular', 'EntityType', 'EvidenceRole',
       'DeviceId', 'Sha256', 'IpAddress', 'Url', 'AccountSid', 'AccountUpn',
       'AccountObjectId', 'AccountName', 'DeviceName', 'NetworkMessageId',
       'EmailClusterId', 'RegistryKey', 'RegistryValueName',
       'RegistryValueData', 'ApplicationId', 'ApplicationName',
       'OAuthApplicationId', 'ThreatFamily', 'FileName', 'FolderPath',
       'ResourceIdName', 'ResourceType', 'Roles', 'OSFamily', 'OSVersion',
       'AntispamDirection', 'SuspicionLevel', 'LastVerdict', 'CountryCode',
       'State', 'City', 'Usage'],
      dtype='object')

In [30]:
test_df['Usage'].head()

,Usage
244231,Public
57535,Public
43951,Public
158655,Public
63950,Private


In [31]:
test_df = test_df.drop(['Usage'], axis=1)

In [32]:
train_df.isnull().sum().compute()

,0
Id,0
OrgId,0
IncidentId,0
AlertId,0
Timestamp,0
DetectorId,0
AlertTitle,0
Category,0
MitreTechniques,27235
IncidentGrade,230


In [33]:
test_df.isnull().sum().compute()

,0
Id,0
OrgId,0
IncidentId,0
AlertId,0
Timestamp,0
DetectorId,0
AlertTitle,0
Category,0
MitreTechniques,4607
IncidentGrade,0


In [34]:
train_df.head()

,Id,OrgId,IncidentId,AlertId,Timestamp,DetectorId,AlertTitle,Category,MitreTechniques,IncidentGrade,...,ResourceType,Roles,OSFamily,OSVersion,AntispamDirection,SuspicionLevel,LastVerdict,CountryCode,State,City
208502,8589935127,283,335,509559,2024-06-11T14:15:54.000Z,0,0,InitialAccess,T1078;T1078.004,TruePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630
57767,635655163043,51,44733,295881,2024-06-03T23:17:54.000Z,39,26,Execution,T1559;T1106;T1059.005,BenignPositive,...,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630
235357,1451698948195,13,6868,10238,2024-06-10T18:15:14.000Z,34,1135,Exfiltration,NaN,BenignPositive,...,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630
21126,816043791031,9,64,112355,2024-06-12T15:28:38.000Z,90,4415,Exfiltration,NaN,BenignPositive,...,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630
128964,1408749274237,16,99016,132381,2024-06-03T20:07:56.000Z,24,21634,Exfiltration,NaN,BenignPositive,...,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630


In [35]:
train_df = train_df.drop(['Id'], axis=1)
test_df = test_df.drop(['Id'], axis=1)

In [36]:
train_df = train_df.drop(['MitreTechniques','ActionGrouped', 'ActionGranular', 'EmailClusterId', 'ThreatFamily', 'ResourceType','Roles','AntispamDirection','SuspicionLevel','LastVerdict'], axis=1)

In [37]:
test_df = test_df.drop(['MitreTechniques','ActionGrouped', 'ActionGranular', 'EmailClusterId', 'ThreatFamily', 'ResourceType','Roles','AntispamDirection','SuspicionLevel','LastVerdict'], axis=1)

In [38]:
train_df = train_df.dropna(subset=['IncidentGrade'])

In [39]:
test_df = test_df.dropna(subset=['IncidentGrade'])

In [40]:
train_df.isnull().sum().compute()

,0
OrgId,0
IncidentId,0
AlertId,0
Timestamp,0
DetectorId,0
AlertTitle,0
Category,0
IncidentGrade,0
EntityType,0
EvidenceRole,0


In [41]:
train_df.info()

<class 'dask_expr.DataFrame'>
Columns: 34 entries, OrgId to City
dtypes: int64(29), string(5)

In [42]:
train_df['Timestamp'] = train_df['Timestamp'].str.extract(r'(\d{4}-\d{2}-\d{2})', expand=False)

In [43]:
train_df['Timestamp'] = dd.to_datetime(train_df['Timestamp'], format='%Y-%m-%d')

In [44]:
test_df['Timestamp'] = test_df['Timestamp'].str.extract(r'(\d{4}-\d{2}-\d{2})', expand=False)

In [45]:
test_df['Timestamp'] = dd.to_datetime(test_df['Timestamp'], format='%Y-%m-%d')

## create day of week column

In [46]:
train_df['Day_of_Week'] = train_df['Timestamp'].dt.day_name()

In [47]:
test_df['Day_of_Week'] = test_df['Timestamp'].dt.day_name()

## split train_df into x and y

In [48]:
# first split features and target
train_x = train_df.drop('IncidentGrade', axis=1)
train_y = train_df['IncidentGrade']

In [49]:
test_x = test_df.drop('IncidentGrade', axis=1)
test_y = test_df['IncidentGrade']

# encoding categorical features

In [50]:
for col in train_x.columns:
    if train_x[col].dtype == 'object' or train_x[col].dtype == 'string':
        train_x[col] = train_x[col].astype('category')

In [51]:
for col in test_x.columns:
    if test_x[col].dtype == 'object' or test_x[col].dtype == 'string':
        test_x[col] = test_x[col].astype('category')

In [52]:
categorical_features = train_x.select_dtypes(include=['category']).columns
categorical_features

Index(['Category', 'EntityType', 'EvidenceRole', 'Day_of_Week'], dtype='object')

In [53]:
train_x_cat = train_x.categorize(columns=categorical_features)
test_x_cat = test_x.categorize(columns=categorical_features)

In [54]:
encoder = DummyEncoder()
train_x = encoder.fit_transform(train_x_cat)
train_x.head()

,OrgId,IncidentId,AlertId,Timestamp,DetectorId,AlertTitle,DeviceId,Sha256,IpAddress,Url,...,EntityType_User,EvidenceRole_Impacted,EvidenceRole_Related,Day_of_Week_Friday,Day_of_Week_Monday,Day_of_Week_Saturday,Day_of_Week_Sunday,Day_of_Week_Thursday,Day_of_Week_Tuesday,Day_of_Week_Wednesday
208502,283,335,509559,2024-06-11,0,0,98799,138268,360606,160396,...,True,True,False,False,False,False,False,False,True,False
57767,51,44733,295881,2024-06-03,39,26,98799,138268,360606,160396,...,True,False,True,False,True,False,False,False,False,False
235357,13,6868,10238,2024-06-10,34,1135,98799,412,360606,160396,...,False,True,False,False,True,False,False,False,False,False
21126,9,64,112355,2024-06-12,90,4415,98799,138268,360606,160396,...,True,True,False,False,False,False,False,False,False,True
128964,16,99016,132381,2024-06-03,24,21634,98799,138268,360606,160396,...,False,True,False,False,True,False,False,False,False,False


In [55]:
# convert only boolean columns to int data type
for col in train_x.columns:
    if train_x[col].dtype == 'bool':
        train_x[col] = train_x[col].astype(int)

In [56]:
test_x = encoder.transform(test_x_cat)

In [57]:
for col in test_x.columns:
    if test_x[col].dtype == 'bool':
        test_x[col] = test_x[col].astype(int)

## SMOTE

In [58]:
from imblearn.over_sampling import SMOTE

In [59]:
train_y.value_counts().compute()

,count
IncidentGrade,
BenignPositive,20406
TruePositive,16766
FalsePositive,10180


In [60]:
combined_df = dd.concat([train_x, train_y], axis=1)

In [61]:
def apply_smote(chunk):
    smote = SMOTE()
    #Exclude datetime columns from X
    X = chunk.select_dtypes(exclude=['datetime']).drop('IncidentGrade', axis=1)
    y = chunk['IncidentGrade']
    X_resampled, y_resampled = smote.fit_resample(X, y)
    return pd.concat([X_resampled, y_resampled], axis=1)

In [62]:
combined_df['IncidentGrade'].isnull().sum().compute()

0

In [63]:
# Apply SMOTE on each chunk
meta = pd.DataFrame(columns=[col for col in combined_df.columns if col != 'Timestamp'])
resampled_chunks = combined_df.map_partitions(apply_smote, meta=meta)

In [64]:
# Combine the resampled chunks
resampled_df = dd.concat([resampled_chunks], axis=0)

In [65]:
test_x = test_x.drop('Timestamp', axis=1)

## split into 80-20 train and test for Evaluation

In [66]:
# split resampled data into train_x and y
train_x = resampled_df.drop('IncidentGrade', axis=1)
train_y = resampled_df['IncidentGrade']

In [67]:
# let's impute the categorical target variable into integer data type by imputation
mapping = {'FalsePositive': 0, 'TruePositive': 1, 'BenignPositive': 2}
train_y = train_y.map(mapping)

In [68]:
test_y = test_y.map(mapping)

In [69]:
# repart_train_x = train_x.repartition(partition_size='100MB')
# repart_train_y = train_y.repartition(partition_size='100MB')

In [70]:
# constant_columns = [col for col in train_x.columns if train_x[col].nunique().compute() == 1]
# constant_columns

In [71]:
train_x_array = train_x.to_dask_array(lengths=True)
train_y_array = train_y.to_dask_array(lengths=True)

In [72]:
# split x and y for train and test
x_train, x_test, y_train, y_test = train_test_split(train_x_array, train_y_array, test_size=0.2, random_state=42)

## Model Evaluation

In [73]:
# from dask.distributed import Client

In [74]:
# client = Client(n_workers = 8, memory_limit='2GB')

In [75]:
models = {
    'Logistic Regression': ParallelPostFit(estimator=LogisticRegression()),
    'Random Forest': ParallelPostFit(estimator=RandomForestClassifier()),
    'Decision Tree': ParallelPostFit(estimator=DecisionTreeClassifier()),
    'XGBoost': ParallelPostFit(estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
}

In [76]:
# wrapped_models = {name: ParallelPostFit(model) for name, model in models.items()}

In [77]:
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [78]:
# for name, model in models.items():
#     print(f"Evaluating {name}...")
#     scores = []
#     for train_index, test_index in skf.split(train_x, train_y):
#         skf_x_train, skf_x_test = train_x.iloc[train_index], train_x.iloc[test_index]
#         skf_y_train, skf_y_test = train_y.iloc[train_index], train_y.iloc[test_index]
#         model.fit(skf_x_train, skf_y_train)
#         y_pred = model.predict(skf_x_test)
#         scores.append(accuracy_score(skf_y_test, y_pred))

#     print(f"{name} Accuracy score: {np.mean(scores):.4f}")

In [79]:
x_train = x_train.astype(np.float64)  # Convert x_train y_train to float64
y_train = y_train.astype(np.float64)

In [80]:
y_test = y_test.astype(np.float64)
x_test = x_test.astype(np.float64)

In [82]:
# for name, model in models.items():
#     print(f"Evaluating {name}...")
#     model.fit(x_train, y_train)
#     score = model.score(x_test, y_test)
#     print(f"{name} Accuracy score: {score}")

# client.close()

In [83]:
# for name, model in models.items():
#     print(f"Evaluating {name}...")
#     model.fit(x_train, y_train) # Fit the model directly on Dask DataFrames
#     score = model.score(x_test, y_test) # Evaluate the model directly on Dask DataFrames
#     print(f"{name} Accuracy score: {score}")

# client.close()

In [84]:
# client.close()